# 8 - Data Wrangling: Join, Combine, and Reshape


In [2]:
import pandas as pd
import numpy as np

## 8.1 Hierarchical Indexing

### MultiIndexing

Hierarchical indexing refers to pandas ability to have objects
indexed in more than one level: Here's some ways of accessing these indices:

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.Series(np.random.uniform(size=9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.159402
   2    0.446368
   3    0.098547
b  1    0.414727
   3    0.081056
c  1    0.051931
   2    0.902408
d  2    0.894399
   3    0.127687
dtype: float64

In [5]:
data.iloc[1]

np.float64(0.44636839745939305)

In [6]:
data.loc['a', 2]

np.float64(0.44636839745939305)

In [7]:
data.loc['b':'d']

b  1    0.414727
   3    0.081056
c  1    0.051931
   2    0.902408
d  2    0.894399
   3    0.127687
dtype: float64

In [8]:
data.loc[['b', 'd']]

b  1    0.414727
   3    0.081056
d  2    0.894399
   3    0.127687
dtype: float64

In [9]:
data.loc[:, 2]

a    0.446368
c    0.902408
d    0.894399
dtype: float64

Hierarchical indexed data can be rearranged into more dimensions
when doing operations. This example can be `unstack()`ed in a DataFrame:

In [10]:
data.unstack()

,1,2,3
a,0.159402,0.446368,0.098547
b,0.414727,NaN,0.081056
c,0.051931,0.902408,NaN
d,NaN,0.894399,0.127687


In [11]:
data.unstack().stack()

a  1    0.159402
   2    0.446368
   3    0.098547
b  1    0.414727
   3    0.081056
c  1    0.051931
   2    0.902408
d  2    0.894399
   3    0.127687
dtype: float64

DataFrames also can have multilevel indexing in columns and index:

In [12]:
frame = pd.DataFrame(np.arange(12).reshape(4,3),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.loc[('a', 2), ('Ohio', 'Red')]

np.int64(4)

We can check the levels of an index with the `nlevel` attribute:

In [14]:
frame.index.nlevels

2

In [15]:
frame.columns.nlevels

2

We can name the indices:

In [16]:
frame.columns.names = ['states', 'colors']
frame.index.names=['key1', 'key2']
frame

states     Ohio     Colorado
colors    Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Finally, we can create the MultiIndex by itself and then reuse it:

In [17]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['states', 'colors'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['states', 'colors'])

### Reordering and Sorting Levels

In [18]:
frame.swaplevel('key1', 'key2')

states     Ohio     Colorado
colors    Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [19]:
frame.swaplevel(0, 1).sort_index(level=0)

states     Ohio     Colorado
colors    Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [20]:
frame.columns.swaplevel(0, 1)

MultiIndex([('Green',     'Ohio'),
            (  'Red',     'Ohio'),
            ('Green', 'Colorado')],
           names=['colors', 'states'])

Data selection is much better on hierarchically sorted indexed objects if the index is lexicographically sorted with the outer most level 
That is, use `sort_index()`!

### Summary statistics by level

We can compute summary statistics by level, for example, with the
`groupby(level=n)` method:

In [21]:
frame.groupby(level='key2').sum()

states  Ohio     Colorado
colors Green Red    Green
key2                     
1          6   8       10
2         12  14       16

In [22]:
frame.groupby(level='colors', axis='columns').mean()

/tmp/ipykernel_17881/2900275161.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level='colors', axis='columns').mean()


colors     Green   Red
key1 key2             
a    1       1.0   1.0
     2       4.0   4.0
b    1       7.0   7.0
     2      10.0  10.0

In [23]:
dataset = pd.DataFrame({
    'Sexo': ['H', 'M', 'M', 'M', 'M', 'H', 'H', 'H', 'M', 'M'],
    'Idade': [53, 72, 54, 27, 30, 40, 58, 32, 44, 51]
})
dataset.groupby('Sexo').mean()

,Idade
Sexo,
H,45.750000
M,46.333333


In [24]:
df = pd.DataFrame(data = {'Fulano': [8, 10, 4, 8, 6, 10, 8],
                          'Sicrano': [7.5, 8, 7, 8, 8, 8.5, 7]}, 
                  index = ['Matemática', 
                           'Português', 
                           'Inglês', 
                           'Geografia', 
                           'História', 
                           'Física', 
                           'Química'])
df.rename_axis('Matérias', axis = 'columns', inplace = True)
(df - df.mean()).abs().mean()

Matérias
Fulano     1.551020
Sicrano    0.469388
dtype: float64

### Indexing with a DataFrame's columns

Pandas allows us to "move" columns to indices with
`df.set_index([columns])`

In [25]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1), "c": ["one", "one", "one", "two", "two", "two", "two"], "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [26]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

We can also do the opposite: move hierarchical
index levels to columns with `df.reset_index()`

In [27]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

There are a number of different combining and merging
operations in pandas. We'll start with the famous *join*
operations from databases, here implemented in the 
`pd.merge(df1, df2)`

### Database-Style DataFrame Joins

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"], "data2": pd.Series(range(3), dtype="Int64")})

df1

In [ ]:
df2

Here we'll do a *many-to-one* join operation, which I
recall from mapping N:1 relations in databases. In that
case, the '1' table is simply added as a column in
the N table:

In [ ]:
pd.merge(df1, df2)

Although we didn't specify which key to merge on, it is
good practice to do so:

In [ ]:
pd.merge(df1, df2, on='key')

If we need to specify different keys for different
tables, we do that with `left_on=` and `right_on=`:

In [ ]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"], "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

The default join is an inner join, which is an intersection
and drops missing keys in both tables. We can also do outer,
left and right joins with the `how=` keyword:

In [ ]:
pd.merge(df1, df2, how='outer')

**Many to many** joins form the cartesian product of the
tables joined. This means that for n keys 'a' found in
table 1, and m keys 'b' found in table 2, there will be
n*m keys 'a' in the resulting table, minimum of 1.

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"], "data2": pd.Series(range(5), dtype="Int64")})

df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on='key', how='right')

We can merge on multiple keys with a list of column names:

In [ ]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"], 
                     "key2": ["one", "two", "one"], 
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"], 
                      "key2": ["one", "one", "one", "two"], 
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

When merging with multiple keys, to think about which keys
will be present in the resulting DataFrame, we can think about
the pairs of keys from each table as being one single key
consisting of a tuple that's being matched against other tuples.

Lastly, when merging DataFrames that have columns with overlapping
names that are NOT the keys being merged on, pandas treats this
by adding suffixes to each dataframe (`_x` to the right, `_y` to the
left one). We can override the suffixes names with the `suffix=['_left',
'_right']` parameter.

### Merging on Index

We can merge on indexes instead of columns with the
`left_index=True` and `right_index=True` parameters.

DataFrames have a `.join(df)` method to simplify joining
on index. This is by default a `left` join, and we can
specify a column from the passed df to join onto.

In [ ]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"], "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

left1

In [ ]:
right1

In [ ]:
left1.join(right1)

In [ ]:
left1.join(right1, on='key')

Lastly, we can pass a list of DataFrames to `join()`
as an alternative to concating lots of times.

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=["a", "c", "e"], columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=["b", "c", "d", "e"], columns=["Missouri", "Alabama"]).astype("Int64")

another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], index=["a", "c", "e", "f"], columns=["New York", "Oregon"])

In [ ]:
left2

In [ ]:
right2

In [ ]:
another

In [ ]:
left2.join([right2, another])

In [ ]:
left2.join([right2, another], how='outer')

### Concatenating Along an Axis

This is inspired from numpy's `concatenate()` method:

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr

In [ ]:
np.concatenate([arr, arr])

In [ ]:
np.concatenate([arr, arr], axis=1)

In pandas we implement this functionality as well,
but considering that the data is labeled and we may
or may not want to use only values with common labels,
or identify the concatenated data in the resulting frame, 
and preserving data. Consider these Series:

In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64") 
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
s1

In [ ]:
s2

In [ ]:
s3

In [ ]:
pd.concat([s1, s2, s3])

We can concat over the 'columns' axis as well.
Concat is by default an outer join:

In [ ]:
pd.concat([s1, s2, s3], axis=1)

We choose the type of concatenation with the
`join=` method.

In [ ]:
pd.concat([s1, s2], axis=1, join='inner')

In [ ]:
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis=1, join='inner')

We can identify which object each label came from
with the `keys=[]` parameter, which will result in
an hierarchically indexed object:

In [ ]:
result = pd.concat([s1, s1, s4], keys=['one', 'two', 'three'])
result

In [ ]:
result.unstack()

Concating over the columns with keys specified turns 
the keys into column labels:

In [ ]:
result2 = pd.concat([s1, s1, s4], axis=1, keys=['one', 'two', 'three'])
result2

Note that the above dataframe is equivalent to the 
unstacked version of concating over the index:

In [ ]:
result2.T

The result is similar when concating dataframes.
The keys will become higher level column labels,
as a way to identify from which dataframe each 
column came from:

In [76]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"], columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"], columns=["three", "four"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [77]:
df2

,three,four
a,5,6
c,7,8


In [78]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

We can achieve the same result (but with arguably
more precision in the naming if each level) by passing
a dictionary to the concat argument. Each dict key will
be a higher level label to identify the dataframe value:

In [79]:
pd.concat({'level1':df1, 'level2':df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

We can name the column levels created with the 
`names=` parameter:

In [80]:
pd.concat({'level1':df1, 'level2':df2}, axis=1, names=['higher', 'lower'])

higher level1     level2     
lower     one two  three four
a           0   1    5.0  6.0
b           2   3    NaN  NaN
c           4   5    7.0  8.0

Lastly, if the index from the rows does not contain
any useful data, we may discard it with the `ignore_index=True`
argument, which will reset the indexes in the resulting DataFrame

In [81]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,-0.056232,2.262075,-0.056935,0.983293
1,0.716770,1.212213,-0.828146,0.257610
2,-0.401198,0.190750,0.135775,1.003178


In [82]:
df2

,b,d,a
0,0.756576,-0.825313,-0.627690
1,1.628949,0.263687,1.153404


In [83]:
pd.concat([df1, df2])

,a,b,c,d
0,-0.056232,2.262075,-0.056935,0.983293
1,0.716770,1.212213,-0.828146,0.257610
2,-0.401198,0.190750,0.135775,1.003178
0,-0.627690,0.756576,NaN,-0.825313
1,1.153404,1.628949,NaN,0.263687


In [84]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.056232,2.262075,-0.056935,0.983293
1,0.716770,1.212213,-0.828146,0.257610
2,-0.401198,0.190750,0.135775,1.003178
3,-0.627690,0.756576,NaN,-0.825313
4,1.153404,1.628949,NaN,0.263687


Lastly, the `verify_integrity=True` parameter
will make the concatenation fail if there are
any duplicates in the objects' indices:

In [85]:
pd.concat([s1, s1], verify_integrity=True, axis=0)

ValueError: Indexes have overlapping values: Index(['a', 'b'], dtype='object')

In [86]:
pd.concat([s1, s1], verify_integrity=True, axis=1)

,0,1
a,0,0
b,1,1


### Combining Data with Overlap

Pandas has a way of using data from a DataFrame
to patch missing values from another. This is done
by the `combine_first()`, which will result in the
union of both dataframes

In [87]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan], "b": [np.nan, 2., np.nan, 6.], "c": range(2, 18, 4)})

df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.], "b": [np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [88]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [89]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 Reshaping and Pivoting

These are operations for rearranging tabular data.

### Reshaping with Hierarchical Indexing

There are two primary actions for reshaping data
arranged in hierarchical indexing (or not!):
`stack()` and `unstack()`

In [99]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Stacking this will turn the number columns into
the inner-most layer of the index:

In [101]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

We can unstack it:

In [102]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Although by default the inner-most level is
unstacked into a column, we can choose which
level is with the `(level=)` keyword and the
int or name corresponding to the unstacked level:

In [103]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [104]:
result.unstack(level='state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if
the unstacked label aren't found in both parent
indices:

In [105]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"]) 

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [106]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


By default, stacking filters missing data to
make the operation more easily reversible. This
can be overridden with `stack(dropna=false)`

In [107]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [109]:
data2.unstack().stack(dropna=False)

/tmp/ipykernel_17881/3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

The level `unstacked` is always the lowest
level in the resulting object:

In [114]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [113]:
df = pd.DataFrame({"left": result, "right": result + 5}, columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [115]:
df.unstack(level='state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

We can also specify the level to be stacked:

In [120]:
df.unstack(level='state').stack(future_stack=True,level='side').sort_index(axis=1)

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting 'Long' to 'Wide' Format

This is how to turn data stacked in a long
format into a wider format, while showing off
the author's ability to deal with time series data:

In [130]:
data = pd.read_csv("../pydata-book/examples/macrodata.csv")
data= data.loc[:, ['year', 'quarter', 'realgdp', 'infl', 'unemp']]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


First he turns 'year' and 'quarter' into a 
PeriodIndex, which will be later discussed,
with `datetime` values at the end of each quarter:

In [131]:
periods = pd.PeriodIndex(year=data.pop('year'),
                         quarter=data.pop('quarter'),
                         name='date')
periods

/tmp/ipykernel_17881/307891108.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop('year'),


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [132]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Then, he selects a subset of columns and give them a name 'item'

In [133]:
data = data.reindex(columns=['realgdp', 'infl', 'unemp'])

In [136]:
data.columns.name='item'
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Finally, he makes the data long by stacking it,
resetting indices and renaming the column containing
the data to 'value'

In [141]:
long_data = data.stack()
long_data

date        item   
1959-01-01  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-04-01  realgdp     2778.801
            infl           2.340
                         ...    
2009-04-01  infl           3.370
            unemp          9.200
2009-07-01  realgdp    12990.341
            infl           3.560
            unemp          9.600
Length: 609, dtype: float64

In [143]:
long_data = long_data.reset_index()
long_data

,date,item,0
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [144]:
long_data = long_data.rename(columns={0:'values'})
long_data

,date,item,values
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


Although this method of table is frequently used
to store data in databases, we main want to untie
this mess with the `.pivot(index=, columns=, values=)` 
method:

In [145]:
pivoted = long_data.pivot(index='date', columns='item', values='values')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


If we had two values columns and omitted the
`value` keyword, the result would be a hierarchical
column dataframe:

In [146]:
long_data['value2'] = np.random.standard_normal(len(long_data))
long_data.head()

,date,item,values,value2
0,1959-01-01,realgdp,2710.349,-1.179245
1,1959-01-01,infl,0.000,0.815686
2,1959-01-01,unemp,5.800,-0.515318
3,1959-04-01,realgdp,2778.801,1.504274
4,1959-04-01,infl,2.340,0.277652


In [147]:
long_data.pivot(index='date', columns='item')

values                     value2                    
item         infl    realgdp unemp      infl   realgdp     unemp
date                                                            
1959-01-01   0.00   2710.349   5.8  0.815686 -1.179245 -0.515318
1959-04-01   2.34   2778.801   5.1  0.277652  1.504274  0.108492
1959-07-01   2.74   2775.488   5.3 -1.921381  0.192182 -0.234432
1959-10-01   0.27   2785.204   5.6  1.889890 -0.803386 -0.239232
1960-01-01   2.31   2847.699   5.2  1.439073  0.668460 -0.467975
...           ...        ...   ...       ...       ...       ...
2008-07-01  -3.16  13324.600   6.0  0.346572 -0.789242 -1.457800
2008-10-01  -8.79  13141.920   6.9  0.957310 -0.814950  1.305411
2009-01-01   0.94  12925.410   8.1  0.307587  0.085604  0.730623
2009-04-01   3.37  12901.504   9.2  0.669901  0.300615  0.014487
2009-07-01   3.56  12990.341   9.6 -0.625212  2.137233 -2.179595

[203 rows x 6 columns]

*pivot* is equivalent to creating a hierarchical index 
using *set_index* followed by an unstack:

In [148]:
unstacked = long_data.set_index(['date', 'item'])
unstacked

values    value2
date       item                        
1959-01-01 realgdp   2710.349 -1.179245
           infl         0.000  0.815686
           unemp        5.800 -0.515318
1959-04-01 realgdp   2778.801  1.504274
           infl         2.340  0.277652
...                       ...       ...
2009-04-01 infl         3.370  0.669901
           unemp        9.200  0.014487
2009-07-01 realgdp  12990.341  2.137233
           infl         3.560 -0.625212
           unemp        9.600 -2.179595

[609 rows x 2 columns]

In [149]:
unstacked = unstacked.unstack(level='item')
unstacked

values                     value2                    
item         infl    realgdp unemp      infl   realgdp     unemp
date                                                            
1959-01-01   0.00   2710.349   5.8  0.815686 -1.179245 -0.515318
1959-04-01   2.34   2778.801   5.1  0.277652  1.504274  0.108492
1959-07-01   2.74   2775.488   5.3 -1.921381  0.192182 -0.234432
1959-10-01   0.27   2785.204   5.6  1.889890 -0.803386 -0.239232
1960-01-01   2.31   2847.699   5.2  1.439073  0.668460 -0.467975
...           ...        ...   ...       ...       ...       ...
2008-07-01  -3.16  13324.600   6.0  0.346572 -0.789242 -1.457800
2008-10-01  -8.79  13141.920   6.9  0.957310 -0.814950  1.305411
2009-01-01   0.94  12925.410   8.1  0.307587  0.085604  0.730623
2009-04-01   3.37  12901.504   9.2  0.669901  0.300615  0.014487
2009-07-01   3.56  12990.341   9.6 -0.625212  2.137233 -2.179595

[203 rows x 6 columns]

### Pivoting "Wide" to "Long" Format

The opposite method from `pivot()` is `melt()`

## Summary:

- We learned multilevel indexing and manipulation by giving names and stacks/unstacking
    - We can `swaplevel(key1, key2)` and `frame.columns.swaplevel(n, m)`
    - If we `groupby(level)`, we can compute summary statistics from a certain level;
    - We can set columns to indices with `set_index([column_list])` and turn them to columns with `reset_index()`
- We learned to combine datasets with `merge()` (on key), `concat()` and `df.join()` (add datasets), `combine_first`(fill holes in datasets)
    - We can merge on indexes too (`right_index=True`)
    - Concating can be done over axis=0 or axis=1
- We reshape with hierarchical indexes by stacking and unstacking selecting the level we want to move;
    - A simpler way and pretty used is to `pivot()` to a wide format and `melt()` into a long format;